<b>
    
<p>
<center>
<font size="5">
Collaborative Filtering 
</font>
</center>
    <center>
<font size="4">
Model-Based Collaborative Filtering Using ALS
</font>
</center>
</p>
   
</b>

<p><h2>Table of Contents<span class="tocSkip"></span></h2></p>

<div class="toc"><ul class="toc-item">
<li><span><a href="#Import"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span></li>
    <li><span><a href="#Import"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import Library</a></span></li>
    <li><span><a href="#LoadData"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load Data</a></span>
        <li><span><a href="#BestModel"><span class="toc-item-num">4&nbsp;&nbsp;</span>Find Best Model</a></span>
        <ul class="toc-item">
            <li><span><a ><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Split Data</a></span></li>
            <li><span><a ><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Build Model</a></span></li>
            <li><span><a ><span class="toc-item-num">4.3&nbsp;&nbsp;</span>ParamGrid Builder and choose Best Model</a></span></li>
            <li><span><a ><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Save&Load Model</a></span></li></ul></li>
    <li><span><a ><span class="toc-item-num">5&nbsp;&nbsp;</span>Build Recommend For User(20 movies)</a></span></li>

### Overview

#### Aim
1.  Sử dụng thuật toán Alternating Least Squares(ALS) để dự đoán rating của người dùng với tất cả các bộ phim 
2. Đưa ra top 20 bộ phim có dự đoán rating cao nhất của từng người dùng

### Import Library

In [1]:
import os
os.environ["SPARK_HOME"] = "/opt/spark"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre"

In [2]:
import findspark
findspark.init()
from pyspark.ml.recommendation import ALSModel
from pyspark.sql import SparkSession,SQLContext
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import explode
from pyspark.sql.functions import mean, col

In [3]:
spark = SparkSession.builder \
    .appName("Project") \
    .master("spark://127.0.0.1:7077") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.cores", "8") \
    .config("spark.driver.extraJavaOptions", "-Xss512m") \
    .config("spark.executor.extraJavaOptions", "-Xss512m") \
    .getOrCreate()


23/05/27 11:33:18 WARN Utils: Your hostname, longcule-Inspiron-7591 resolves to a loopback address: 127.0.1.1; using 192.168.1.40 instead (on interface enx68e43b306c5a)
23/05/27 11:33:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/27 11:33:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Load Data

In [4]:
ratings_df = spark.read.csv("hdfs://127.0.0.1:9900/user/ratings.csv", header=True, inferSchema=True)
movies = spark.read.csv("hdfs://127.0.0.1:9900/user/movies.csv", header=True, inferSchema=True)

In [5]:
ratings_df.show(5)
ratings_df.drop('timestamp')

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    307|   3.5|1256677221|
|     1|    481|   3.5|1256677456|
|     1|   1091|   1.5|1256677471|
|     1|   1257|   4.5|1256677460|
|     1|   1449|   4.5|1256677264|
+------+-------+------+----------+
only showing top 5 rows



DataFrame[userId: int, movieId: int, rating: double]

Sprase data rating

In [6]:

numerator = ratings_df.count()

users = ratings_df.select("userId").distinct().count()
movies = ratings_df.select("movieId").distinct().count()

denominator = users * movies

sparsity = 1 - (numerator*1.0 / denominator)
print ("Sparsity: "), sparsity

Sparsity: 


(None, 0.9981816370521007)

### Find Best Model

#### Split data (Train/test - 80/20)

In [7]:
(training_data, test_data) = ratings_df.randomSplit([0.8, 0.2], seed = 3834)

#### Build Model

In [8]:

def train_ALS(train_data, validation_data, num_iters, reg_params, ranks):
    min_error = float('inf')
    best_rank = -1
    best_regularization = 0
    best_model = None
    
    for num_iter in num_iters: 
        for rank in ranks:
            for reg_param in reg_params:
                als = ALS(rank=rank, maxIter=num_iter, regParam=reg_param, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
                model = als.fit(train_data)

                predictions = model.transform(validation_data)
                evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
                rmse_error = evaluator.evaluate(predictions)
                print("Root-mean-square error = " + str(rmse_error))

                print('rank: {}, num_iter: {}, reg_param: {} validation RMSE is {}'.format(rank, num_iter, reg_param, rmse_error))
                if rmse_error < min_error:
                    min_error = rmse_error
                    best_rank = rank
                    best_regularization = reg_param
                    best_model = model
    
    print('The best model has {} latent factors and regularization = {}'.format(best_rank, best_regularization))
    return best_model

#### ParamGridBuilder Choose Best Model

In [9]:
num_iterations = [50]
ranks = [100]
reg_params = [0.05]
import time

start_time = time.time()
final_model = train_ALS(training_data, test_data, num_iterations, reg_params, ranks)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

Root-mean-square error = 0.7784632834708709
rank: 100, num_iter: 50, reg_param: 0.05 validation RMSE is 0.7784632834708709
The best model has 100 latent factors and regularization = 0.05
Total Runtime: 4457.57 seconds


#### Save & Load Model

In [10]:
final_model.save("ALS_Movielens_27M")
final_model = ALSModel.load("ALS_Movielens_27M")

In [11]:
userRecs = final_model.recommendForAllUsers(20)

userRecs.show()

/opt/spark/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    26|[{86955, 5.729708...|
|    27|[{134853, 4.86921...|
|    28|[{156846, 5.15952...|
|    31|[{142871, 5.21941...|
|    34|[{188189, 4.75715...|
|    44|[{8533, 4.837329}...|
|    53|[{108180, 4.57013...|
|    65|[{131166, 4.78489...|
|    76|[{123497, 4.96401...|
|    78|[{2571, 4.879174}...|
|    81|[{60983, 5.271923...|
|    85|[{2502, 3.9420466...|
|   101|[{144448, 4.66748...|
|   108|[{86237, 5.278370...|
|   115|[{2609, 4.8907127...|
|   126|[{171011, 4.38200...|
|   133|[{2795, 4.0398917...|
|   137|[{92136, 4.767501...|
|   148|[{93040, 4.541998...|
|   155|[{103751, 5.50779...|
+------+--------------------+
only showing top 20 rows



In [12]:
userRecs.write.mode('overwrite').parquet('model')
recommendation = spark.read.parquet('model')

### Make Recommend For User (20 movies)

In [15]:
def get_recommendations(user_id):
    recs = recommendation.filter(col("userId") == user_id).select("recommendations")
    recs = recs.select(explode(col("recommendations")).alias("rec")).select("rec.movieId", "rec.rating")
    item_list = recs.orderBy(col("rating").desc()).select("movieId").rdd.flatMap(lambda x: x).collect()
    return item_list

get_recommendations(1) 

[1197,
 120807,
 105189,
 61013,
 160289,
 7361,
 134849,
 90464,
 183947,
 159817,
 2502,
 1449,
 1288,
 176601,
 6375,
 3911,
 101850,
 7141,
 175879,
 171011]